In [2]:
import pandas as pd

In [3]:
ccle_mutation="/Volumes/Samsung_T5/Depmap_CCLE/CCLE_mutations_2019Oct28.csv"
annotation_file="/Volumes/Samsung_T5/Depmap_CCLE/sample_info.csv"
phospho_file="/Volumes/Samsung_T5/UDAI_PCA_BISSAN/LATEST_ANALYSIS/AllData_includingNonsignificant_ContinuousSens_ReadyforAnalysis.txt"

annot_read=pd.read_csv(annotation_file)
mutation_read=pd.read_csv(ccle_mutation,low_memory=False)
phospho_read=pd.read_csv(phospho_file,sep="\t",low_memory=False)
phospho_read.insert(0,'Identifier',(phospho_read['CellLine']+"_"+phospho_read['Drug']),True)

In [4]:
deptocelllinename_dict={}
celllinenametodep_dict={}

for i in annot_read.index:
    deptocelllinename_dict[annot_read.at[i,'DepMap_ID']]=annot_read.at[i,'stripped_cell_line_name']
    celllinenametodep_dict[annot_read.at[i,'stripped_cell_line_name']]=annot_read.at[i,'DepMap_ID']

In [4]:
mutation_read.head()

,Hugo_Symbol,Entrez_Gene_Id,NCBI_Build,Chromosome,Start_position,End_position,Strand,Variant_Classification,Variant_Type,Reference_Allele,...,ExAC_AF,CGA_WES_AC,SangerWES_AC,SangerRecalibWES_AC,RNAseq_AC,HC_AC,RD_AC,WGS_AC,Variant_annotation,DepMap_ID
0,VPS13D,55187,37,1,12359347,12359347,+,Nonsense_Mutation,SNP,C,...,NaN,NaN,34:221,34:213,NaN,NaN,NaN,NaN,damaging,ACH-000001
1,AADACL4,343066,37,1,12726308,12726322,+,In_Frame_Del,DEL,CTGGCGTGACGCCAT,...,NaN,NaN,9:0,53:193,NaN,NaN,NaN,28:32,other non-conserving,ACH-000001
2,IFNLR1,163702,37,1,24484172,24484172,+,Silent,SNP,G,...,NaN,NaN,118:0,118:0,10:0,NaN,NaN,18:0,silent,ACH-000001
3,TMEM57,55219,37,1,25785018,25785019,+,Frame_Shift_Ins,INS,-,...,NaN,NaN,NaN,NaN,6:28,NaN,NaN,NaN,damaging,ACH-000001
4,ZSCAN20,7579,37,1,33954141,33954141,+,Missense_Mutation,SNP,T,...,NaN,NaN,27:61,28:62,NaN,NaN,NaN,NaN,other non-conserving,ACH-000001


In [5]:
not_preferred_mutations=["Silent","Intron","IGR","5'Flank","3'UTR","nan","5'UTR"]
celllines_intheexperiment=["NCIH1373","NCIH1437","NCIH1563","NCIH1568","NCIH1650","NCIH1693","NCIH1734","NCIH1755","NCIH1781","NCIH1792","NCIH1793","NCIH1838","NCIH1944","NCIH1975","NCIH1993","NCIH2030","NCIH2087","NCIH2122","NCIH2228","NCIH2291","NCIH23","NCIH2347","NCIH3122","NCIH358","NCIH441","NCIH522","NCIH820","HCC2935","HCC4006","HCC44","HCC827","PC9","SKLU1","A549","CORL105"]
celllines_withhardIDs=["H1373","H1437","H1563","H1568","H1650","H1693","H1734","H1755","H1781","H1792","H1793","H1838","H1944","H1975","H1993","H2030","H2087","H2122","H2228","H2291","H23","H2347","H3122","H358","H441","H522","H820"]
important_genelist=["KRAS","EGFR","BRAF","NRAS","MAP2K1","PIK3CA","AKT1","PTEN","STK11","MET","ERBB2","ERBB3","FGFR1","FGFR2","FGFR3","TP53"]

In [6]:
selected_mutations=mutation_read[~mutation_read['Variant_Classification'].isin(not_preferred_mutations)].reset_index(drop=True)
selected_mutations['DepMap_ID']=selected_mutations['DepMap_ID'].map(deptocelllinename_dict)
selected_mutations=selected_mutations[selected_mutations['DepMap_ID'].isin(celllines_intheexperiment)].reset_index(drop=True)
genelist=selected_mutations['Hugo_Symbol'].unique()
selected_mutations

,Hugo_Symbol,Entrez_Gene_Id,NCBI_Build,Chromosome,Start_position,End_position,Strand,Variant_Classification,Variant_Type,Reference_Allele,...,ExAC_AF,CGA_WES_AC,SangerWES_AC,SangerRecalibWES_AC,RNAseq_AC,HC_AC,RD_AC,WGS_AC,Variant_annotation,DepMap_ID
0,C1orf170,0,37,1,915845,915845,+,Missense_Mutation,SNP,C,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10:17,other non-conserving,HCC827
1,CASZ1,54897,37,1,10699463,10699463,+,Missense_Mutation,SNP,C,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7:6,other non-conserving,HCC827
2,IGSF21,84966,37,1,18554419,18554420,+,Frame_Shift_Ins,INS,-,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4:18,damaging,HCC827
3,HSPG2,3339,37,1,22166035,22166035,+,Missense_Mutation,SNP,C,...,0.000008,NaN,NaN,NaN,89:59,NaN,NaN,13:9,other non-conserving,HCC827
4,FGR,2268,37,1,27942323,27942323,+,Missense_Mutation,SNP,C,...,NaN,NaN,NaN,3:2,NaN,6:4,NaN,10:12,other non-conserving,HCC827
5,EPB41,2035,37,1,29313995,29313995,+,Missense_Mutation,SNP,C,...,NaN,NaN,75:79,75:77,25:36,NaN,NaN,9:17,other non-conserving,HCC827
6,CSMD2,114784,37,1,34190992,34190992,+,Missense_Mutation,SNP,C,...,0.000008,NaN,11:24,11:23,NaN,NaN,NaN,4:5,other non-conserving,HCC827
7,B4GALT2,8704,37,1,44445557,44445557,+,De_novo_Start_OutOfFrame,SNP,C,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8:35,damaging,HCC827
8,PTCH2,8643,37,1,45288958,45288958,+,Missense_Mutation,SNP,C,...,NaN,NaN,16:63,16:59,NaN,NaN,NaN,NaN,other non-conserving,HCC827
9,LDLRAD1,388633,37,1,54474686,54474686,+,Nonsense_Mutation,SNP,C,...,0.000041,NaN,135:46,135:46,NaN,NaN,NaN,26:6,damaging,HCC827


In [107]:
idlist=phospho_read['Identifier']

In [77]:
phospho_read.head()

,Identifier,Mutation_Status,Drug,CellLine,ContinousSensitivity,AKT308,AKT473,ATF2,AXL,bCATENIN,...,PTEN,Rb,RET,RPS6,SRC,STAT1,STAT3,STAT5A/B,TSC2,VEGFR2
0,A549_AUY922,KRAS_mutant,AUY922,A549,0.183428,1.723378,1.382587,0.525571,0.280380,0.198916,...,0.393595,0.504422,0.477841,-0.884453,-0.317668,0.053360,0.096103,0.096103,0.353928,0.030612
1,CORL105_AUY922,Wild_type,AUY922,CORL105,0.037535,1.692594,1.174550,0.223668,0.201581,0.248734,...,-0.088096,-0.263271,0.299786,-0.045362,0.205679,-0.002174,-0.273953,-0.026717,-0.092733,0.235747
2,H1373_AUY922,KRAS_mutant,AUY922,H1373,0.182295,0.063956,-0.261689,0.343909,0.348465,0.229829,...,-0.251487,0.088860,0.170395,-0.210226,-0.212316,0.099487,-0.116119,0.355864,-0.923411,-0.329428
3,H1437_AUY922,Wild_type,AUY922,H1437,0.390510,0.033780,-1.139547,-0.596475,-0.272233,-0.665473,...,-2.146095,-0.485348,-0.389199,-1.208994,-0.706712,-0.571827,-0.228272,-0.298370,-1.466721,-0.574631
4,H1563_AUY922,Wild_type,AUY922,H1563,0.283711,-0.708120,-0.385513,0.096124,-0.066325,-0.009997,...,-2.295826,0.036525,-0.193557,-1.010941,-0.026612,0.041017,-0.104874,0.407649,-1.843038,0.001695


In [133]:
newframe=pd.DataFrame(columns=genelist)
newframe.insert(0,"ContinousSensitivity",phospho_read['ContinousSensitivity'],True)
newframe.insert(0,"CellLine",phospho_read['CellLine'],True)
newframe.insert(0,"Drug",phospho_read['Drug'],True)
newframe.insert(0,"Mutation_Status",phospho_read['Mutation_Status'],True)
newframe.insert(0,"Identifier",phospho_read['Identifier'],True)


In [134]:
columnindex=0
celllines=newframe['CellLine'].unique()
cellline=""
for cell_string in celllines:
    cellline=cell_string
    #cellline=identifier.split("_")[0]
    cellline_indices = [i for i, value in enumerate(newframe['CellLine']) if value == cellline]
    if cellline in celllines_withhardIDs:
        cellline="NCI"+cellline
    
    print(cell_string)
    for gene in genelist:
        columnindex=newframe.columns.get_loc(gene)
        tempresult=selected_mutations[(selected_mutations.DepMap_ID==cellline) & (selected_mutations.Hugo_Symbol==gene)]
        for number in cellline_indices:
            if tempresult.shape[0] ==0:
                newframe.iloc[number,columnindex]=0
            else:
                newframe.iloc[number,columnindex]=1
                #print(columnindex)
    

A549
CORL105
H1373
H1437
H1563
H1568
H1650
H1693
H1734
H1755
H1781
H1792
H1793
H1838
H1944
H1975
H1993
H2030
H2087
H2122
H2228
H2291
H23
H2347
H3122
H358
H441
H522
H820
HCC2935
HCC4006
HCC44
HCC827
PC9
SKLU1


In [130]:
res_list = [i for i, value in enumerate(newframe['CellLine']) if value == "A549"]

[0, 35, 70, 105, 140, 175, 210]

In [138]:
newframe.to_csv("SOCRATES_celllines_w_CCLEMutation.csv",index=False)

In [167]:
for gen in important_genelist:
    if gen not in newframe.columns:
        important_genelist.remove(gen)
important_genelist.insert(0,"ContinousSensitivity")
important_genelist.insert(0,"CellLine")
important_genelist.insert(0,"Drug")
important_genelist.insert(0,"Mutation_Status")
important_genelist.insert(0,"Identifier")

In [168]:
frame_w_important_mutations=newframe[important_genelist]

In [169]:
frame_w_important_mutations.to_csv("SOCRATES_celllines_w_CCLEMutation_onlyIMPORTANTMUTATIONS.csv",index=False)

In [7]:
selected_mutations[selected_mutations['Hugo_Symbol']=="ERBB2"]

,Hugo_Symbol,Entrez_Gene_Id,NCBI_Build,Chromosome,Start_position,End_position,Strand,Variant_Classification,Variant_Type,Reference_Allele,...,ExAC_AF,CGA_WES_AC,SangerWES_AC,SangerRecalibWES_AC,RNAseq_AC,HC_AC,RD_AC,WGS_AC,Variant_annotation,DepMap_ID
2914,ERBB2,2064,37,17,37880997,37880998,+,In_Frame_Ins,INS,-,...,NaN,NaN,36:4,44:14,100:1,11:0,827:35,NaN,other non-conserving,NCIH1781
14599,ERBB2,2064,37,17,37872661,37872661,+,Missense_Mutation,SNP,G,...,NaN,NaN,3:7,3:7,63:143,5:1,236:479,6:19,other non-conserving,NCIH1793
15654,ERBB2,2064,37,17,37868282,37868282,+,Missense_Mutation,SNP,A,...,NaN,NaN,234:118,231:118,570:255,525:295,NaN,NaN,other non-conserving,NCIH1563
